# xhycom for HYCOM OPeNDAP and plotting demonstration
**Author: Jun Sasaki, Coded on July 23, 2020, Updated on August 7, 2020**<br>
- See xhycom/utils.py, run_hycom.py
- When encountering `urlopen error [Errno -2] Name or service not known`, reduce zoom.
- To avoid decoding problem in time coordinate, var "tau" is deleted. If "tau" is necessary, manual decoding is required.

In [ ]:
from xhycom import utils as xh
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
import xarray as xr
from netCDF4 import num2date, date2num
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import Stamen
from cartopy.io.img_tiles import OSM
from cartopy.mpl.ticker import LatitudeFormatter,LongitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
#%matplotlib widget  ### fig.savefig() does not work

# HYCOM data download
## Downloading one time and creating netcdf
- Enforce `opendap=False` if already downloaded
- In general, use run_opendap()

In [ ]:
opendap=False  ### True: Download HYCOM data using OPeNDAP, False: Use existing netcdf
extent = (139.2, 140.2, 34.8, 35.8)  ### (lon_min, lon_max, lat_min, lat_max)
time = '2012-10-28 18:00:00'
ncfile="hycom_tokyo_bay.nc"

if opendap:
    ds = xh.run_hycom_gofs3_1_region_ymdh(extent=extent, time=time)
    ds.to_netcdf(ncfile, mode="w")
ds = xr.open_dataset(ncfile)
ds

## Same as above but using xh.run_opendap()
- Enfoece `opendap=False` if already downloaded.
- `xh.run_opendap(extent, time_start, time_end=None, dtime=3, tz='utc')`
- Netcdf file name is automatically generated as "hycom_" + datetime + ".nc" 
- This method should be used in general.

In [ ]:
opendap=False  ### True: Download HYCOM data using OPeNDAP
extent = (139.2, 140.2, 34.8, 35.8)  ### (lon_min, lon_max, lat_min, lat_max)
time = '2012-10-30 18:00:00'
if opendap:
    xh.run_opendap(extent=extent, time_start=time)

## Downloading and creating multi-time netcdf
- `xh.run_opendap(extent, time_start, time_end=None, dtime=3, tz='utc')`
- Netcdf file name is automatically generated as "hycom_" + datetime + ".nc" 
- This method should be used in general.

In [ ]:
opendap=False  ### True: Download HYCOM data using OPeNDAP, False: Use existing netcdf
extent = (139.2, 140.2, 34.8, 35.8)
time_start, time_end, dtime = ('2012-10-28 12:00:00', '2012-10-29 00:00:00', 3)  ### dtime (int): time interval in hours
if opendap:
    xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime)

# Plotting HYCOM netcdf with [matplotlib](https://matplotlib.org/#)
- NetCDF is loaded as xarray.Dataset.
- matplotlib is used for static plotting, which is suitable for publication.

## Loading netcdf into xarray.Dataset

In [ ]:
ncfile = "hycom_2012-10-29_00.nc"
ds = xr.open_dataset(ncfile)
ds
#ds.salinity
#ds["salinity"]
#ds[["salinity"]]

In [ ]:
### Examples of indexing and slicing
#ds["salinity"]
#ds.salinity
#ds.salinity.isel(depth=0, time=0)
#ds.salinity.isel(depth=slice(0,3), time=[0])
ds.salinity.sel(depth=[0.1, 15.0], method="nearest")

## Simple plotting
Very simple but the coastline is ugly, x- and y- axes labels are missing, and the legend is too long by default.

In [ ]:
### Start setting by users
depth = 0 # depth index in int
extent = (139.2, 140.2, 34.8, 35.8)  ### (lon_min, lon_max, lat_min, lat_max)
figsize = (6,6)
png = 'hycom_tokyo_bay1.png'
## Default font size
plt.rcParams['font.size'] = 12
### End setting by users

central_longitude = np.mean(extent[0:2])
#proj = ccrs.LambertConformal(central_longitude=central_longitude, central_latitude=central_latitude)
proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)
ds.salinity.isel(depth=depth).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())
ax.coastlines()
fig.savefig(png, dpi=600, bbox_inches='tight')

## Simple plotting with minimum axes adjustment

In [ ]:
### Start setting by users
depth = 0 # depth index in int
extent = (139.2, 140.2, 34.8, 35.8)
figsize = (6,5)
png = 'hycom_tokyo_bay2.png'
## Default font size
plt.rcParams['font.size'] = 12
## Adjust surorunding margins
lon_min, lon_max = extent[0]-0.1, extent[1]+0.15
lat_min, lat_max = extent[2]-0.1, extent[3]+0.1
### End setting by users

central_longitude = np.mean(extent[0:2])
#proj = ccrs.LambertConformal(central_longitude=central_longitude, central_latitude=central_latitude)
proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)
ds.salinity.isel(depth=depth).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())
ax.set_extent([lon_min,lon_max,lat_min,lat_max], crs=ccrs.PlateCarree())
ax.coastlines()
gl=ax.gridlines(draw_labels=True, linewidth=0.5, color='k', alpha=0.8)
gl.right_labels=False
gl.top_labels=False
fig.savefig(png, dpi=600, bbox_inches='tight')

## Simple plotting with manual axes adjustment

In [ ]:
### Start settings by users
extent = (139.2, 140.2, 34.8, 35.8)
depth = 0 # depth index in int
figsize = (6,5)
png = 'hycom_tokyo_bay3.png'
## Default font size
plt.rcParams['font.size'] = 12
## Adjust surrounding margins
lon_min, lon_max = extent[0]-0.1, extent[1]+0.15
lat_min, lat_max = extent[2]-0.1, extent[3]+0.1
## Ticks intervals for lon and lat axes
dlon, dlat = (0.4, 0.2)
### End settings by users

central_longitude = np.mean(extent[0:2])
xticks = np.arange(lon_min, lon_max, dlon)
yticks = np.arange(lat_min, lat_max, dlat)
proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)
ds.salinity.isel(depth=depth).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.coastlines()
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=1, color='k', alpha=0.8)
gl.xlocator = mticker.FixedLocator(xticks)
gl.ylocator = mticker.FixedLocator(yticks)
ax.set_xticks(xticks,crs=ccrs.PlateCarree())
ax.set_yticks(yticks,crs=ccrs.PlateCarree())
latfmt=LatitudeFormatter()
lonfmt=LongitudeFormatter(zero_direction_label=True)
ax.xaxis.set_major_formatter(lonfmt)
ax.yaxis.set_major_formatter(latfmt)
ax.axes.tick_params(labelsize=12)
fig.savefig(png, dpi=600, bbox_inches='tight')

## Simple plotting with manual axes adjustment and background map (tiler)

In [ ]:
### Start settings by users
# Background map
tiler = Stamen('terrain-background')
#tiler = OSM()
zoom=9
extent = (139.2, 140.2, 34.8, 35.8)
figsize = (6,5)
depth = 0 # depth index in int
png = 'hycom_tokyo_bay4.png'
## Default font size
plt.rcParams['font.size'] = 14
## Override axes label size
axes_label_size = 12
## Adjust surronding margins
lon_min, lon_max = extent[0]-0.1, extent[1]+0.15
lat_min, lat_max = extent[2]-0.1, extent[3]+0.1
## Ticks intervals
dlon, dlat = (0.4, 0.2)
### End settings by users

xticks = np.arange(lon_min, lon_max, dlon)
yticks = np.arange(lat_min, lat_max, dlat)
central_longitude = np.mean(extent[0:2])
proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)
ds.salinity.isel(depth=0).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())
ax.set_extent([lon_min,lon_max,lat_min,lat_max], crs=ccrs.PlateCarree())
#gl=ax.gridlines(draw_labels=True, xlocs=xlocs, ylocs=ylocs, linewidth=0.5, color='k', alpha=0.8)
gl=ax.gridlines(draw_labels=True, xlocs=xticks, ylocs=yticks, linestyle=':', linewidth=1, color='k', alpha=0.8)
#fig.canvas.draw()
gl.right_labels=False
gl.top_labels=False
gl.xlabel_style={'size':axes_label_size}
gl.ylabel_style={'size':axes_label_size}
gl.xlocator = mticker.FixedLocator(xticks)
ax.add_image(tiler, zoom)
fig.savefig(png, dpi=600, bbox_inches='tight')

## Function ax_lonlat_axes for adjusting axes
`ax_lonlat_axes(ax, extent, grid_linestyle=':', grid_linewidth=0.5, grid_color='k', \
                grid_alpha=0.8, xticks=None, yticks=None, label_size=12, tiler=None, zoom=8)`<br>
### Args:
-    ax(Axis)                 : gets current Axis
-    extent (tuple)           : (lon_min, lon_max, lat_min, lat_max)
-    grid_linestyle(str)      : linestyle (default: ':')
-    grid_linewidth(float/int): linewidth (default: 0.5)
-    grid_color(str)          : color (default: 'k')
-    grid_alpha(float)        : opacity (default: 0.8)
-    xticks(list)             : list of xticks (default: None)
-    yticks(list)             : list of yticks (default: None)
-    label_size(int)          : label size in pt (default: 12)
-    tiler(cartopy.io.img_tiles):
-    zoom(int)                : zoom in tiler
### Returns
-    ax(Axis)

## Plotting using ax_lonlat_axes and background map

In [ ]:
### Start settings by users
extent = (139.2, 140.2, 34.8, 35.8)
tiler = Stamen('terrain-background')  ### tiler = Stamen('terrain-background')/OSM()/None 
zoom = 9
## Default font size
plt.rcParams['font.size'] = 14
## Override axes label size
axes_label_size = 12
grid_linewidth=1
figsize = (6,5)
depth = 0 # depth index in int
png = 'hycom_tokyo_bay5.png'
## Adjust surronding margins
lon_min, lon_max = extent[0]-0.1, extent[1]+0.15
lat_min, lat_max = extent[2]-0.1, extent[3]+0.1
## Tick intervals
dlon, dlat = (0.4, 0.2)
### End settings by users

xticks = np.arange(lon_min, lon_max, dlon)
yticks = np.arange(lat_min, lat_max, dlat)
central_longitude, central_latitude = np.mean(extent[0:2]), np.mean(extent[2:4])

proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)

ds.salinity.isel(depth=0).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())

ax = xh.ax_lonlat_axes(ax, extent=(lon_min, lon_max, lat_min, lat_max), \
               grid_linewidth=grid_linewidth, grid_color='k', \
               xticks=xticks, yticks=yticks, label_size=axes_label_size, tiler=tiler, zoom=zoom)
fig.savefig(png, dpi=600, bbox_inches='tight')

# Interactive plotting with [hvPlot](https://hvplot.holoviz.org/#)
- Useful for checking data values of `(x, y, z)` interactively.
- Slower than matplotlib; may not be suitable for publication.
- [Geoviews](https://geoviews.org/#) may also be useful.

[hvPlot geographic data](https://hvplot.holoviz.org/user_guide/Geographic_Data.html) and [hvPlot options](https://hvplot.holoviz.org/user_guide/Customization.html)<br>

In [ ]:
### Geoviews
'''
import warnings
warnings.filterwarnings('ignore')
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts
gv.extension('bokeh')
gv.output(size=100)

gvds=gv.Dataset(ds.salinity.isel(time=0))

qmesh = gvds.to(gv.QuadMesh, ['lon', 'lat'], dynamic=True).redim.range(salinity=(34.0, 34.3))
qmesh.opts(colorbar=True, cmap='jet', projection=ccrs.PlateCarree(), width=400, height=300 ,tools=['hover']) \
* gf.coastline.options(scale='10m')
'''

## Plan view
`xarray.Dataset.hvplot.quadmesh(x, y, z, project, coastline, frame_height, cmap)`<br>
- [cmap](https://matplotlib.org/gallery/color/colormap_reference.html?highlight=cmap): matplotlib colormap (for details see [here](https://matplotlib.org/tutorials/colors/colormaps.html))
- `coastline = ['10m'/'50m'/'110m']`
- When `tiles` is specified (e.g., `tiles="OSM"`), axes labels are ignored -> maybe a bug.


In [ ]:
### When tiles is set (e.g., tiles="OSM"), axes labels are ignored -> maybe bug.
z='salinity'
cmap='magma_r' ## _r : reversed 
frame_height=300
project = ccrs.PlateCarree()
ds.hvplot.quadmesh(x='lon', y='lat', z=z, project=project, tiles=None, coastline='10m', frame_height=frame_height, cmap=cmap)

## Vertical sectional views
`xarray.Dataset.hvplot.quadmesh(x, y, z, flip_yaxis, frame_width, frame_height, cmap)`<br>
- `flip_yaxis=True`: reverse yaxis

In [ ]:
x = 'lon'
z = 'salinity'
cmap = 'magma_r'
frame_width = 300
frame_height = 200

ds.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_width=frame_width, frame_height=frame_height, cmap=cmap)

In [ ]:
x = 'lat'
ds.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_width=frame_width, frame_height=frame_height, cmap=cmap)

# Interactive time series plotting with hvPlot
- Using dask for large data handling
- See [Readng and writing files - xarray](http://xarray.pydata.org/en/stable/io.html)

## Loading netcdf files

In [ ]:
ncfiles = 'hycom_2012*.nc'
dsts = xr.open_mfdataset(ncfiles, parallel=True, concat_dim="time", data_vars='minimal', coords='minimal', compat='override')
dsts

## Plan view
`xarray.Dataset.hvplot.quadmesh(x, y, z, project, coastline, frame_height, cmap)`

In [ ]:
z = 'salinity'
project = ccrs.PlateCarree()
frame_height = 300
cmap = 'magma_r'

dsts.hvplot.quadmesh(x='lon', y='lat', z=z, project=project, coastline='10m', frame_height=frame_height, cmap=cmap)

## Vertical sectional views

In [ ]:
x = 'lon'
z = 'salinity'
frame_width  = 300
frame_height = 200
cmap = 'magma_r'

dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, cmap=cmap)

In [ ]:
x = 'lat'
dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, cmap=cmap)

# Create GIF animation with matplotlib

In [ ]:
from matplotlib.animation import FuncAnimation
# This is needed to display graphics calculated outside of jupyter notebook
from IPython.display import HTML, display

In [ ]:
dsts.time

In [ ]:
### Start settings by users
ncfiles = 'hycom_2012*.nc'
## Select indexes for depth and time
depth=0
time=0
extent = (139.2, 140.2, 34.8, 35.8)
tiler = Stamen('terrain-background')  ### tiler = Stamen('terrain-background')/OSM()/None 
zoom = 9
## Default font size
plt.rcParams['font.size'] = 11
## Override axes label size
axes_label_size = 11
grid_linewidth=1
figsize = (6,5)
png = 'hycom_tokyo_bay5.png'
## Adjust surronding margins
lon_min, lon_max = extent[0]-0.1, extent[1]+0.15
lat_min, lat_max = extent[2]-0.1, extent[3]+0.1
## Tick intervals
dlon, dlat = (0.4, 0.2)
### End settings by users

xticks = np.arange(lon_min, lon_max, dlon)
yticks = np.arange(lat_min, lat_max, dlat)
central_longitude, central_latitude = np.mean(extent[0:2]), np.mean(extent[2:4])

proj = ccrs.PlateCarree(central_longitude=central_longitude)
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)

fig.subplots_adjust(left=0.1, bottom=0.05, right=0.95, top=0.95, wspace=None, hspace=None)

ax = xh.ax_lonlat_axes(ax, extent=(lon_min, lon_max, lat_min, lat_max), \
           grid_linewidth=grid_linewidth, grid_color='k', \
           xticks=xticks, yticks=yticks, label_size=axes_label_size, tiler=tiler, zoom=zoom)
cax = dsts.salinity.isel(depth=depth, time=0).plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree(),\
                                             vmin=34.0, vmax=34.3, levels=100, cbar_kwargs={'shrink':0.85})

def update(i):
    #print(i)
    cax.set_array(dsts.salinity.isel(depth=depth, time=i).values.flatten())
    ax.set_title("Depth = " + str(dsts['depth'].values[depth]) + "  Time = " + str(dsts.coords['time'].values[i])[0:16])
ani = FuncAnimation(fig, update, frames=3, interval=200, blit=False, repeat=True)
ani.save("anime.gif", writer="pillow", dpi=200)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
ani.save('./Python_Animation_04.mp4')
display(HTML("<video controls><source src='./Python_Animation_04.mp4' type='video/mp4'></video>"))